# Data Exploration

This notebook describes the data exploration steps.

## Install dependencies

In [3]:
%pip install -r data/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Load data

In [48]:
import pandas as pd

df = pd.read_sql_table('offenses', 'sqlite:///data/data.sqlite')

### Dataset
Below, you can see the dataset. The available columns are:
- `exceedance`: The exceedance of the speed limit in km/h
- `datetime`: The date and time of the offense
- `lat`: The latitude of the offense
- `lon`: The longitude of the offense
- `temperature`: The temperature at the offense location at the offense time
- `precipitation`: The precipitation at the offense location at the offense time
- `wind speed`: The wind speed at the offense location at the offense time

In [3]:
df.head(10)

,exceedance,datetime,lat,lon,temperature,precipitation,wind speed
0,6,2018-01-01 00:00:29,50.951697,6.981953,7.4,0.8,28.4
1,17,2018-01-01 00:01:11,50.947906,6.941059,7.4,0.8,28.4
2,6,2018-01-01 00:06:44,50.951697,6.981953,7.4,0.8,28.4
3,8,2018-01-01 00:08:34,50.936374,6.935985,7.4,0.8,28.4
4,21,2018-01-01 00:12:08,50.947906,6.941059,7.4,0.8,28.4
5,9,2018-01-01 00:14:17,50.947906,6.941059,7.4,0.8,28.4
6,16,2018-01-01 00:19:01,50.947906,6.941059,7.4,0.8,28.4
7,12,2018-01-01 00:20:21,50.936374,6.935985,7.4,0.8,28.4
8,52,2018-01-01 00:20:45,50.947906,6.941059,7.4,0.8,28.4
9,6,2018-01-01 00:21:11,50.947906,6.941059,7.4,0.8,28.4


### Data exploration
To get a first impression of the data, I executed Pandas [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function on the dataset. The output can be seen below.

In [7]:
df.describe()

,datetime,lat,lon,temperature,precipitation,wind speed
count,665635,665635.000000,665635.000000,665635.000000,665635.000000,662923.000000
mean,2019-08-11 14:11:30.047725824,50.952222,6.950637,13.105690,0.086298,12.980099
min,2018-01-01 00:00:29,50.849154,6.846078,-11.900000,0.000000,0.000000
25%,2018-10-02 20:00:20,50.947906,6.941059,6.900000,0.000000,7.900000
50%,2019-05-12 07:37:09,50.947906,6.941059,12.300000,0.000000,11.900000
75%,2020-05-25 12:51:20,50.951697,6.976871,18.900000,0.000000,16.900000
max,2021-12-31 23:57:25,51.046593,7.085497,40.800000,32.700000,72.000000
std,NaN,0.024807,0.034965,7.827018,0.522428,6.646244


## Correlations
To find correlations, firstly I calculated the Pearson correlation coefficient between the weather data and the speeding offenses.

### Pearson correlation coefficient
Calculating the pearson correlation coefficient for the exceedance columns and the temperature, precipitation and wind speed columns


In [25]:
df['exceedance'].corr(df['temperature'], method='pearson')


0.025064393831251693

In [24]:
df['exceedance'].corr(df['precipitation'], method='pearson')

-0.014784174460152422

In [26]:
df['exceedance'].corr(df['wind speed'], method='pearson')

0.0038510883940356634

All of the above values are rather low and don't indicate a correlation between the weather and the speeding offenses. So up next I tried to find a correlation between the weather and the speeding offenses by counting the number of offenses per weather condition.

### Counting offenses per weather condition
In this section I counted the number of offenses per weather condition. The weather conditions are grouped into 6 categories, each representing an interval of the temperature, precipitation and wind speed values. The categories are:

- `cold`: temperature < 5°C
- `warm`: temperature >= 25°C
- `dry`: precipitation < 0.5mm
- `wet`: precipitation >= 2mm
- `calm`: wind speed < 5km/h
- `windy`: wind speed >= 25km/h

In [45]:
all = len(df)
count_cold = len(df[(df['temperature'] < 5)])
print('Percentage of exceedances at cold temperatures: ' + str(count_cold / all * 100) + '%')

count_warm = len(df[(df['temperature'] >= 25)])
print('Percentage of exceedances at warm temperatures: ' + str(count_warm / all * 100) + '%')

count_dry = len(df[(df['precipitation'] < 0.5)])
print('Percentage of exceedances at dry weather: ' + str(count_dry / all * 100) + '%')

count_wet = len(df[(df['precipitation'] >= 2)])
print('Percentage of exceedances at wet weather: ' + str(count_wet / all * 100) + '%')

count_calm = len(df[(df['wind speed'] < 5)])
print('Percentage of exceedances at calm weather: ' + str(count_calm / all * 100) + '%')

count_windy = len(df[(df['wind speed'] >= 20)])
print('Percentage of exceedances at windy weather: ' + str(count_windy / all * 100) + '%')

Percentage of exceedances at cold temperatures: 15.14073028010847%
Percentage of exceedances at warm temperatures: 7.800521306722152%
Percentage of exceedances at dry weather: 95.25881301313784%
Percentage of exceedances at wet weather: 1.0043041606886656%
Percentage of exceedances at calm weather: 7.340058740901545%
Percentage of exceedances at windy weather: 14.99365267751846%


The above results shows that with 95% almost all offenses happen in the `dry` category. This is not surprising since the weather in cologne is mostly dry. So I tried to find a correlation between the weather and the speeding offenses by counting the number of offenses per weather condition in relation to the number of days with that weather condition.

### Counting offenses per weather condition in relation to the number of days with that weather condition

To find a more meaningful relation between the weather and the speeding offenses I counted the number of offenses per weather condition in relation to the number of days with that weather condition

In [47]:
# count number of days in cologne with percipation below 0.5mm
from meteostat import Point, Hourly
from datetime import datetime

start = datetime(2017, 1, 1)
end = datetime(2020, 12, 31)

cologne = Point(50.9381, 6.9569)
data = Hourly(cologne, start, end)
data = data.fetch()

count_cold_hours = len(data[(data['temp'] < 5)])
count_warm_hours = len(data[(data['temp'] >= 25)])
count_dry_hours = len(data[(data['prcp'] < 0.5)])
count_wet_hours = len(data[(data['prcp'] >= 2)])
count_calm_hours = len(data[(data['wspd'] < 5)])
count_windy_hours = len(data[(data['wspd'] >= 20)])

print('Percentage of offenses at cold temperatures: ' + str(count_cold_hours / count_cold * 100) + '%')
print('Percentage of offenses at warm temperatures: ' + str(count_warm_hours / count_warm * 100) + '%')
print('Percentage of offenses at dry weather: ' + str(count_dry_hours / count_dry * 100) + '%')
print('Percentage of offenses at wet weather: ' + str(count_wet_hours / count_wet * 100) + '%')
print('Percentage of offenses at calm weather: ' + str(count_calm_hours / count_calm * 100) + '%')
print('Percentage of offenses at windy weather: ' + str(count_windy_hours / count_windy * 100) + '%')

Percentage of offenses at cold temperatures: 5.793693318251275%
Percentage of offenses at warm temperatures: 3.670820253067041%
Percentage of offenses at dry weather: 5.269399882663907%
Percentage of offenses at wet weather: 4.637247569184742%
Percentage of offenses at calm weather: 6.359245159441647%
Percentage of offenses at windy weather: 4.315501538029919%


These results do not show any significant difference between the `dry` and `wet` categories anymore and neither reveal any other correlation between the weather and the speeding offenses. This indicates that the `dry` category is overrepresented in the data and that the weather does not have a significant influence on the speeding offenses.